# Data Cleaning

## Text Extraction

In [1]:
%pip install pandas requests beautifulsoup4 openpyxl

Note: you may need to restart the kernel to use updated packages.


In [36]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

# load the Excel file with URLs
df = pd.read_excel('web_scraping_urls.xlsx')

# function to clean text
def clean_text(text):
    # regex to match non-printable characters
    clean_text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    return clean_text

# function to extract and clean text from a URL
def extract_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        # clean the text before returning
        return clean_text(text)
    except requests.RequestException as e:
        return str(e)

# define batch size and output file name
batch_size = 10
output_filename = 'all_extracted_texts_2.xlsx'

# initialize the ExcelWriter to append without overwriting previous data
with pd.ExcelWriter(output_filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    total_urls = len(df['URLs'])

    # process each batch
    for start in range(2210, total_urls, batch_size):
        end = start + batch_size
        print(f"processing batch from URL {start+1} to {min(end, total_urls)}")
        batch_data = {'URL': df['URLs'][start:end], 'Quarter': df['Quarter'][start:end], 'Extracted Text': []}

        for index, url in enumerate(df['URLs'][start:end], start=start+1):
            print(f"processing URL {index}/{total_urls}: {url}")
            text = extract_text(url)
            batch_data['Extracted Text'].append(text)
            time.sleep(2)  # sleep for 2 seconds between requests
            print("extraction successful for this URL")

        # convert batch data to DataFrame and append to the Excel file
        batch_df = pd.DataFrame(batch_data)
        # append batch data to the Excel file
        batch_df.to_excel(writer, sheet_name='Extracted Text', index=False, header=not writer.sheets, startrow=writer.sheets['Extracted Text'].max_row if 'Extracted Text' in writer.sheets else 0)

        print(f"batch from URL {start+1} to {min(end, total_urls)} saved.")

print("all URLs processed and saved to the same Excel file.")


processing batch from URL 2211 to 2220
processing URL 2211/4913: https://www.singstat.gov.sg/-/media/files/publications/reference/newsletter/ssnmar2015.ashx
extraction successful for this URL
processing URL 2212/4913: https://www.marketscreener.com/quote/stock/ARMSTRONG-INDUSTRIAL-CORP-6492231/
extraction successful for this URL
processing URL 2213/4913: https://www.shareinvestor.com/fundamental/factsheet.html?counter=B61.SI
extraction successful for this URL
processing URL 2214/4913: https://www.buynewlaunchnow.com/
extraction successful for this URL
processing URL 2215/4913: https://www.todayonline.com/singapore/jtc-improves-industrial-property-indices-better-reflect-market-landscape
extraction successful for this URL
processing URL 2216/4913: https://www.straitstimes.com/singapore/manufacturing-jobs-most-at-risk-ntuc-warns
extraction successful for this URL
processing URL 2217/4913: https://www.todayonline.com/singapore/singapores-economy-grew-29-2014
extraction successful for this 

KeyboardInterrupt: 

In [35]:
# initialize the ExcelWriter to append without overwriting previous data
with pd.ExcelWriter(output_filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    total_urls = len(df['URLs'])

    start = 2200
    end = 2209
    batch_data = {'URL': df['URLs'][start:end], 'Quarter': df['Quarter'][start:end], 'Extracted Text': []}

    for index, url in enumerate(df['URLs'][start:end], start=start+1):
        print(f"processing URL {index}/{total_urls}: {url}")
        text = extract_text(url)
        batch_data['Extracted Text'].append(text)
        time.sleep(2)  # sleep for 2 seconds between requests
        print("extraction successful for this URL")

    # convert batch data to DataFrame and append to the Excel file
    batch_df = pd.DataFrame(batch_data)
    # append batch data to the Excel file
    batch_df.to_excel(writer, sheet_name='Extracted Text', index=False, header=not writer.sheets, startrow=writer.sheets['Extracted Text'].max_row if 'Extracted Text' in writer.sheets else 0)

    print(f"batch from URL {start+1} to {min(end, total_urls)} saved.")

print("all URLs processed and saved to the same Excel file.")


processing URL 2201/4913: https://www.asianinvestor.net/article/weekly-roundup-of-people-news-march-27/395796
extraction successful for this URL
processing URL 2202/4913: https://focus.kornferry.com/wp-content/uploads/2015/02/The-Global-Talent-Crunch.pdf
extraction successful for this URL
processing URL 2203/4913: https://www.bloomberg.com/profile/company/7731288Z:SP
extraction successful for this URL
processing URL 2204/4913: https://www.slideshare.net/slideshow/singapore-malaysia-economic-analysis/45567957
extraction successful for this URL
processing URL 2205/4913: https://www.china-briefing.com/news/using-baidu-chinas-largest-search-engine/
extraction successful for this URL
processing URL 2206/4913: https://knowledge.insead.edu/economics-finance/whats-behind-september-stock-market-blues
extraction successful for this URL
processing URL 2207/4913: https://www.cdnetworks.com/
extraction successful for this URL
processing URL 2208/4913: https://www.99.co/singapore/neighbourhoods/juro